In [2]:
from main import init_system
#from pathselection import joinpathselection
#from integrating_civilizer import *
from api.apiutils import Relation
api, reporting = init_system("/Users/emansour/elab/DAGroup/DataCivilizer/Aurum-GitHub/aurum-datadiscovery/models/mitdw/")

Loading: */Users/emansour/elab/DAGroup/DataCivilizer/Aurum-GitHub/aurum-datadiscovery/models/mitdw/*

### Help Menu

You can use the system through an **API** object. API objects are returnedby the *init_system* function, so you can get one by doing:

***your_api_object = init_system('path_to_stored_model')***

Once you have access to an API object there are a few concepts that are useful to use the API. **content** refers to actual values of a given field. For example, if you have a table with an attribute called __Name__ and values *Olu, Mike, Sam*, content refers to the actual values, e.g. Mike, Sam, Olu.

**schema** refers to the name of a given field. In the previous example, schema refers to the word__Name__ as that's how the field is called.

Finally, **entity** refers to the *semantic type* of the content. This is in experimental state. For the previous example it would return *'person'* as that's what those names refer to.

Certain functions require a *field* as input. In general a field is specified by the source name (e.g. table name) and the field name (e.g. attribute name). For example, if we are interested in finding content similar to the one of the attribute *year* in the table *Employee* we can provide the field in the following way:

field = ('Employee', 'year') # field = [<source_name>, <field_name>)
Took 1.2120800018310547 to load all data


In [4]:
kw = "employee"
res = api.table_name_search(kw)
res.print_tables()

Employee_directory.csv


In [3]:
table = api.drs_from_table("Employee_directory.csv")
res = api.similar_content_to(table)
res.print_columns()

Hit(nid='3805293829', db_name='MITDW', source_name='Zip_usa.csv', field_name='Zip Code', score=0.916708984375)
Hit(nid='1144772871', db_name='MITDW', source_name='Warehouse_users.csv', field_name='Title', score=1)
Hit(nid='180097543', db_name='MITDW', source_name='Employee_directory.csv', field_name='Directory Full Name', score=1)
Hit(nid='3367778836', db_name='MITDW', source_name='Warehouse_users.csv', field_name='Unit Name', score=1)
Hit(nid='3819846166', db_name='MITDW', source_name='Drupal_employee_directory.csv', field_name='Hr Org Unit Title', score=1)
Hit(nid='1510843162', db_name='MITDW', source_name='Warehouse_users.csv', field_name='Unit Id', score=0.9720405932558384)
Hit(nid='2291421469', db_name='MITDW', source_name='Hr_org_unit.csv', field_name='Hr Department Name', score=1)
Hit(nid='1252981536', db_name='MITDW', source_name='Sdo_crs_compound.csv', field_name='Cmpd Horiz Srid', score=0.9962470512545571)
Hit(nid='3924554025', db_name='MITDW', source_name='Drupal_employee_di

In [4]:
field = api.drs_from_raw_field(('mitdwh', 'Employee_directory.csv', 'Primary Title'))
res = api.similar_content_to(field)
res.pretty_print_columns()

DB: mitdwh               TABLE: Warehouse_users.csv            FIELD: Title                         
DB: mitdwh               TABLE: Employee_directory.csv         FIELD: Directory Title               
DB: mitdwh               TABLE: Se_person.csv                  FIELD: Position Title                
DB: mitdwh               TABLE: Drupal_employee_directory.csv  FIELD: Primary Title                 
DB: mitdwh               TABLE: Drupal_employee_directory.csv  FIELD: Directory Title               


In [5]:
res.why_id('1239902562')

The result does not exist


# Story

We have 2400 tables in MIT data warehouse and among them 160 are semi-public. In this demo we will show how to find all the employees in MIT working in the degree granting department.

# 1-Discovering Relevant Tables

### I want to find tables about employees

In [2]:
res = api.table_name_search('Employee')
res.print_tables()

Employee_directory.csv


In [6]:
res.pretty_print_columns()

DB: mitdwh               TABLE: Employee_directory.csv         FIELD: Office Location               
DB: mitdwh               TABLE: Employee_directory.csv         FIELD: Department Name               
DB: mitdwh               TABLE: Employee_directory.csv         FIELD: Krb Name                      
DB: mitdwh               TABLE: Employee_directory.csv         FIELD: Personal Url                  
DB: mitdwh               TABLE: Employee_directory.csv         FIELD: Name Known By                 
DB: mitdwh               TABLE: Employee_directory.csv         FIELD: Full Name                     
DB: mitdwh               TABLE: Employee_directory.csv         FIELD: Office Phone                  
DB: mitdwh               TABLE: Employee_directory.csv         FIELD: Krb Name Uppercase            
DB: mitdwh               TABLE: Employee_directory.csv         FIELD: Email Address                 
DB: mitdwh               TABLE: Employee_directory.csv         FIELD: Last Name            

### I want to find tables that contain information about degree granting

In [3]:
res = api.schema_name_search("degree")
res.pretty_print_columns()

DB: mitdwh               TABLE: Sis_department.csv             FIELD: Is Degree Granting            
DB: mitdwh               TABLE: Hr_faculty_roster.csv          FIELD: Terminal Degree               
DB: mitdwh               TABLE: Student_degree_program.csv     FIELD: Degree Desc                   
DB: mitdwh               TABLE: Sis_course_description.csv     FIELD: Is Degree Granting            
DB: mitdwh               TABLE: Student_degree_program.csv     FIELD: Degree Code                   
DB: mitdwh               TABLE: Student_degree_program.csv     FIELD: Degree Type                   
DB: mitdwh               TABLE: Student_degree_program.csv     FIELD: Degree Weight                 
DB: mitdwh               TABLE: Student_degree_program.csv     FIELD: Degree Last Activity Date     
DB: mitdwh               TABLE: Sis_course_description.csv     FIELD: Default Ultimate Degree       
DB: mitdwh               TABLE: Student_degree_program.csv     FIELD: Degree Desc Short    

Not only I'm interested in tables with attributes that are related to **degree granting**, but also on those whose semantics are closer to **department**. I can capture that with a refined discovery query.

In [4]:
res1 = api.schema_name_search("degree")
res2 = api.table_name_search("department")
res1.set_table_mode()
res2.set_table_mode()
res3 = api.intersection(res1, res2)
res3.print_tables()

Sis_department.csv


**Sis_department.csv** seems a good candidate.

# 2-Finding Join Paths

I have found two tables that are relevant to my query: Who employees work in degree granting departments? To run my query, however, I'd like to have a **view** of both tables. Can I join them together?

In [9]:
t1 = "Employee_directory.csv"
t2 = "Sis_department.csv"

I'm going to create a discovery query that finds join paths between two given tables. And I'm going to wrap up that function into another, for convenience.

In [5]:
def joinPaths(table1, table2):
    drs_t1 = api.drs_from_table(table1)
    drs_t2 = api.drs_from_table(table2)
    drs_t1.set_table_mode()
    drs_t2.set_table_mode()
    res = api.paths_between(drs_t1, drs_t2, Relation.PKFK, max_hops=2)
    #res = api.paths_between(drs_t1, drs_t2, Relation.CONTENT_SIM, max_hops=1)
    return res

In [11]:
t1 = "Employee_directory.csv"
t2 = "Sis_department.csv"
res = joinPaths(t1, t2)
format_join_paths(res)

[(mitdwh.Employee_directory.csv.Department Number 2036610506 0, None), (mitdwh.Sdo_crs_projected.csv.Srid 4008886731 0, mitdwh.Sdo_crs_projected.csv.Srid 4008886731 0.9997029997029997), (mitdwh.Sis_department.csv.Dept Budget Code 1239902562 0.9997029997029997, None)]
[(mitdwh.Employee_directory.csv.Department Number 2036610506 0, None), (mitdwh.Fac_organization.csv.Major Org 47923891 0, mitdwh.Fac_organization.csv.Hr Department Code Old 444650383 0.9419354838709677), (mitdwh.Sis_department.csv.Dept Budget Code 1239902562 0.9872611464968153, None)]
[(mitdwh.Employee_directory.csv.Department Number 2036610506 0, None), (mitdwh.Fac_organization.csv.Major Org 47923891 0, mitdwh.Fac_organization.csv.Major Org 47923891 0.9872611464968153), (mitdwh.Sis_department.csv.Dept Budget Code 1239902562 0.9872611464968153, None)]
[(mitdwh.Employee_directory.csv.Department Number 2036610506 0, None), (mitdwh.Fac_organization.csv.Major Org 47923891 0, mitdwh.Fac_organization.csv.Organization Number 8155

In [6]:
format_join_paths(res)

PATH: 0
degree - degree
 
Student_degree_program.csv - Degree Code
 
PATH: 1
degree - degree
 
Sis_course_description.csv - Default Ultimate Degree
 
PATH: 2
degree - degree
 
Student_degree_program.csv - Degree Last Activity Date
 
PATH: 3
degree - degree
 
Sis_course_description.csv - Is Degree Granting
 
PATH: 4
degree - degree
 
Student_degree_program.csv - Degree Type
 
PATH: 5
degree - degree
 
Hr_faculty_roster.csv - Terminal Degree
 
PATH: 6
degree - degree
 
Student_degree_program.csv - Degree Desc
 
PATH: 7
degree - degree
 
Student_degree_program.csv - Degree Desc Short
 
PATH: 8
degree - degree
 
Sis_department.csv - Is Degree Granting
 
PATH: 9
degree - degree
 
Student_degree_program.csv - Degree Weight
 


There are 4 different join paths! Does anyone work? What are the differences?

# 3 - Selecting Join Paths

### There are 4 different join paths! Which join path should I use? What are the differences?

The join path selection module selects one join path from perhaps many, which maximizes the estimated benefit in terms of the cardinalities and the cleanliness


In [7]:
projections = dict()
projections['Employee_directory.csv'] = ['Full Name', 'Office Phone', 'Primary Title', 'Department Name']
projections['Sis_department.csv'] = ['Is Degree Granting', 'School Name']

(corpus, jps) = join_path_selection(api, res, projections)

[degree.degree.degree 1 -1, mitdwh.Student_degree_program.csv.Degree Code 3823748160 3.3972297]


FileNotFoundError: [Errno 2] No such file or directory: '/Users/emansour/elab/DAGroup/DataCivilizer/Demo/cidr-demo/cidr-demo/datasets/mitdwh_exp/degree'

# 4 - Cleaning the Join Path with a Budget 
### If I have some budget, which cell to clean first?

The join path cleaning module uses a given budget of B user repairing actions to maximize the quality of the view by cleaning the most impact cells in the source table first

In [27]:
budget = 10
join_path = 3
res = clean_with_budget(jps, corpus, budget, join_path)

Table	Row	Col	Impact	 Value
Employee_directory.csv	1169	10	0.967521	
Employee_directory.csv	9510	10	0.967521	
Employee_directory.csv	1169	11	0.965677	Unknown
Employee_directory.csv	1169	9	0.965677	Not available
Employee_directory.csv	9510	11	0.965677	Unknown
Employee_directory.csv	9510	9	0.965677	Not available
Employee_directory.csv	9510	7	0.960665	
Employee_directory.csv	5117	7	0.960111	9999999999
Employee_directory.csv	6082	7	0.960111	9999999999
Employee_directory.csv	10178	7	0.960059	1111111111



In [28]:
peek_results(10, res)

Full Name	Office Phone	Primary Title	Department Name	Is Degree Granting	School Name
Knight Terry W	6174522922	Professor	Department of Architecture	Y	Architecture and Planning
Dutta Arindam	6172531432	Associate Professor	Department of Architecture	Y	Architecture and Planning
Nagakura Takehiko	6172530781	Associate Professor	Department of Architecture	Y	Architecture and Planning
Porter William L	6176206332	Professor Emeritus	Department of Architecture	Y	Architecture and Planning
Caso Renee A	6172537792	Academic Programs Manager	Department of Architecture	Y	Architecture and Planning
Stewart Cynthia	6172534408	Academic Administrator	Department of Architecture	Y	Architecture and Planning
Friedman David Hodes	6172588438	Professor Emeritus	Department of Architecture	Y	Architecture and Planning
Simunovic Anne W	6172534412	Assistant to the Department Head	Department of Architecture	Y	Architecture and Planning
Wodiczko Krzysztof	6173246289	Professor Emeritus	Department of Architecture	Y	Architect